In [3]:
#importer les librairies
import pandas as pd
import re
import sqlparse
import sqlparser
import pyparsing


# Création du dataframe

In [6]:
#créer le dataframe à partir d'un fichier csv
df=pd.read_csv("sql_requetes.csv", sep=";")

In [7]:
#lire toutes les infos du df
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.max_colwidth', None)

In [8]:
#lire les 10 premières lignes du dataframe
df.head(10)

projet  \
0    ATP   
1    ATP   
2    ATP   
3    ATP   
4    ATP   
5    ATP   
6    ATP   
7    PLE   
8    PLE   
9    PLE   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

## Test du sqlparse - ne traite pas la récupération des noms de table

In [16]:
# Divise une chaîne contenant deux instructions SQL :
raw = 'SELECT * FROM question WHERE q_id IN (SELECT r_fk_question_id FROM reponse WHERE r_date_ajout BETWEEN  2013-01-01  AND  2013-12-31 );SELECT * FROM Table'
statements = sqlparse.split(raw)
statements

['SELECT * FROM question WHERE q_id IN (SELECT r_fk_question_id FROM reponse WHERE r_date_ajout BETWEEN  2013-01-01  AND  2013-12-31 );',
 'SELECT * FROM Table']

In [15]:
# Formater la première déclaration :
first = statements[0]
print(sqlparse.format(first, reindent=True, keyword_case='upper'))

SELECT *
FROM question
WHERE q_id IN
    (SELECT r_fk_question_id
     FROM reponse
     WHERE r_date_ajout BETWEEN 2013-01-01 AND 2013-12-31 );


In [16]:
# Analyser une instruction SQL :
parsed = sqlparse.parse('select * from foo')[0]
parsed.tokens

[<DML 'select' at 0x2D326273340>,
 <Whitespace ' ' at 0x2D326272BC0>,
 <Wildcard '*' at 0x2D3262A8400>,
 <Whitespace ' ' at 0x2D3262A83A0>,
 <Keyword 'from' at 0x2D3262A84C0>,
 <Whitespace ' ' at 0x2D3262A8520>,
 <Identifier 'foo' at 0x2D326214F50>]

## Test sqlparser

In [23]:
def extract_tables(sql):
    tables = set()
    
    # Analyser la requête SQL pour obtenir un objet Token
    tokens = sqlparse.parse(sql)
    
    # Vérifier s'il y a des erreurs de syntaxe
    if not tokens:
        raise Exception('extract_tables: SQL invalide.')

    # Utiliser le premier objet Token (la première instruction SQL)
    root_token = tokens[0]
    
    for item in root_token.tokens:
        if isinstance(item, sqlparse.sql.Identifier):
            tables.add(item.get_real_name())
        elif isinstance(item, sqlparse.sql.IdentifierList):
            # Appel récursif pour les identifiants imbriqués
            tables.update(extract_tables(item))
        elif isinstance(item, sqlparse.sql.Function):
            # Gérer les fonctions pouvant référencer des tables
            tables.update(extract_tables(item))

    return tables




In [25]:
result=extract_tables(raw)
print(result)

{'question'}


## Test d'un nouvelle regex - non fonctionnelle pour toutes les requetes

In [9]:
def extract_tables_from_query(query):
    # Utilisation de sqlparse pour analyser la requête SQL
    parsed = sqlparse.parse(query)

    # Initialisation d'une liste pour stocker les noms de table
    tables = []

    # Fonction récursive pour extraire les tables des sous-requêtes
    def extract_tables_recursive(tokens, ignore_aliases=False):
        for token in tokens:
            if isinstance(token, sqlparse.sql.Identifier):
                # Ignorer les alias de colonne à l'intérieur des sous-requêtes
                if not ignore_aliases:
                    tables.append(str(token.get_real_name()))
            elif isinstance(token, sqlparse.sql.IdentifierList):
                for identifier in token.get_identifiers():
                    extract_tables_recursive([identifier], ignore_aliases)
            elif isinstance(token, sqlparse.sql.Function) and token.get_real_name().upper() == 'FROM':
                # Vérifier si c'est une sous-requête
                if '(' in token.value and ')' in token.value:
                    # Extraire les tables de la sous-requête sans prendre en compte les alias
                    subquery = token.value[token.value.find('(')+1:token.value.rfind(')')]
                    extract_tables_recursive(sqlparse.parse(subquery)[0].tokens, ignore_aliases=True)
                else:
                    # Ce n'est pas une sous-requête, extraire les tables normalement
                    for identifier in token.get_identifiers():
                        extract_tables_recursive([identifier], ignore_aliases)
            elif isinstance(token, sqlparse.sql.Parenthesis):
                # Appel récursif pour traiter les sous-requêtes imbriquées
                extract_tables_recursive(token.tokens, ignore_aliases)

    # Parcours des éléments analysés pour extraire les noms de table
    for stmt in parsed:
        extract_tables_recursive(stmt.tokens)

    return tables

In [10]:
# Exemple d'utilisation avec votre requête
query = "Select * FROM T1,T2 Where T1.ID=T2.FK_ID"
query2="select * from (select nom, adresse from Immeuble where id=1) as Koudalou"
query3="SELECT 	TDI_D01 as TDI_D01, 	TDI_D02 as TDI_D02, 	TDI_D03 as TDI_D03, 	TDI_D04 as TDI_D04, 	TDI_D05 as TDI_D05, 	TDI_D06 as TDI_D06, 	TDI_D07 as TDI_D07, 	null as TDI_D08, 	TDI_D09 as TDI_D09, 	TDI_D10 as TDI_D10, 	TDI_D11 as TDI_D11, 	TDI_D12 as TDI_D12, 	null as TDI_D13, 	null as TDI_D14, 	null as TDI_D15, 	null AS TDI_D16, 	null AS TDI_D17, 	null AS TDI_D18, 	null AS TDI_D19, 	null AS TDI_D20, 	null AS TDI_D21, 	null AS TDI_D22, 	null AS TDI_D23, 	null AS TDI_D24, 	null AS TDI_D25, 	null AS TDI_D26, 	null AS TDI_D27, 	null AS TDI_D28, 	null AS TDI_D29, 	null AS TDI_D30, 	TDI_I01 as TDI_I01, 	TDI_I02 as TDI_I02, 	TDI_I03 as TDI_I03, 	TDI_I04 AS TDI_I04, 	NULL AS TDI_I05, 	NULL AS TDI_I06, 	NULL AS TDI_I07, 	NULL AS TDI_I08, 	NULL AS TDI_I09, 	NULL AS TDI_I10, 	NULL AS TDI_I11, 	NULL AS TDI_I12, 	NULL AS TDI_I13, 	NULL AS TDI_I14, 	NULL AS TDI_I15, 	NULL AS TDI_I16, 	NULL AS TDI_I17, 	NULL AS TDI_I18, 	NULL AS TDI_I19, 	NULL AS TDI_I20 FROM (  SELECT     to_char(CTX.CONT_DATE_CREATION,'yyyyMMdd') AS TDI_D01,     41 as TDI_D02,     CASE WHEN (SUM(CASE WHEN (NVL(THT.ADRC_COM_CONCESS,'N') = 'O') THEN 1 ELSE 0 END)>=1 AND SUM(CASE WHEN (NVL(THT.ADRC_COM_CONCESS,'N') <> 'O') THEN 1 ELSE 0 END)=0) THEN 'T' ELSE         (CASE WHEN (SUM(CASE WHEN (NVL(THT.ADRC_COM_CONCESS,'N') = 'O') THEN 1 ELSE 0 END)=0 AND SUM(CASE WHEN (NVL(THT.ADRC_COM_CONCESS,'N') <> 'O') THEN 1 ELSE 0 END)>=1) THEN 'HT' ELSE            (CASE WHEN (SUM(CASE WHEN (NVL(THT.ADRC_COM_CONCESS,'N') = 'O') THEN 1 ELSE 0 END)>=1 AND SUM(CASE WHEN (NVL(THT.ADRC_COM_CONCESS,'N') <> 'O') THEN 1 ELSE 0 END)>=1) THEN 'MIXTE' ELSE 			'NRE' END) END) END AS  TDI_D03,      NVL(TO_CHAR(ACT.CLI_CHRONO),'NRE') AS TDI_D04,      'C-'||TO_CHAR(CTX.CONT_CHRONO_CONTRAT) as TDI_D05,      NVL(TO_CHAR(SOUS.SOUS_ID), 'NRE') as TDI_D06,      NVL(TO_CHAR(EDC.CPT_ID), 'NRE') as TDI_D07,      'NRE' AS TDI_D09,      NVL(UPPER(ACT.CLI_E_MAIL),'NRE') AS TDI_D10,      NVL(TO_CHAR(CSO.CONSO_CONSO_ANN),'NRE') as TDI_D11,      CTX.CONT_CHRONO_DOSSIER AS TDI_D12,     1 AS TDI_I01,     CASE WHEN CTX.CONT_ETAT_DOSSIER  =  'ENCRS' THEN 1 ELSE 0 END AS TDI_I02,     COUNT(DISTINCT ACT.CLI_CHRONO) AS TDI_I03, 	CASE WHEN ACT.CLI_CHRONO is not null  THEN 1 ELSE 0 END AS TDI_I04      FROM FUS_CONTENTIEUX CTX     LEFT JOIN SIM_GCC_CLIENTS CL ON (CTX.CONT_CHRONO_CLIENT=CL.CLI_CHRONO)           LEFT JOIN (             select                  c.con_chrono,  				CL.CLI_CHRONO,  				ECTR_CODE_ETAT,  				actif,  				dvl.date_debut,  				dvl.date_fin, 				CLI_E_MAIL             from SIM_GCD_CONTRAT C                 INNER JOIN DVL_GCD_ETATS_CONTRAT DVL ON (C.CON_CHRONO=DVL.CON_CHRONO)                 INNER JOIN SIM_GCC_CLIENTS CL ON (CL.CLI_CHRONO=C.CLI_CHRONO AND (CLI_FAMILLE='DOM' OR CLI_CATEGORIE='NDOM'))             where DVL.ECTR_CODE_ETAT IN ('A', 'F') AND CL.CLI_CHRONO IS NOT NULL 			) ACT ON (ACT.CLI_CHRONO=CL.CLI_CHRONO AND CTX.CONT_DATE_CREATION BETWEEN ACT.DATE_DEBUT AND ACT.DATE_FIN)      	LEFT JOIN SIM_GCT_ADR_EN_CLAIR A ON (CL.CLI_ADR_CHRONO=A.ADRC_CHRONO)     LEFT JOIN SIM_GCD_CONTRAT C ON (CONT_CHRONO_CONTRAT=C.CON_CHRONO) 	 	     LEFT JOIN ( 			SELECT  				SIM_GCD_CONTRAT.CON_CHRONO AS CON_CHRONO,  				MAX(SIM_GCD_COND_ADMIN.CADM_TYPE_PAIEMENT) AS TYPE_PAIEMENT 			FROM SIM_GCD_CONTRAT 				INNER JOIN SIM_GCD_ENT_PREST ON (SIM_GCD_CONTRAT.CON_CHRONO=SIM_GCD_ENT_PREST.CON_CHRONO) 				INNER JOIN SIM_GCD_COND_ADMIN ON (SIM_GCD_COND_ADMIN.CADM_CHRONO=SIM_GCD_ENT_PREST.CADM_CHRONO) 			GROUP BY SIM_GCD_CONTRAT.CON_CHRONO 			) TP ON (C.CON_CHRONO = TP.CON_CHRONO) 			     LEFT JOIN SIM_GCC_COMMUNES CO ON (CO.COM_CODE = A.ADRC_COM_CODE)     LEFT JOIN ODS_COMMUNES OC ON (OC.CODE_INSEE = nvl(CO.COM_CODE_INSEE,'NRE'))     LEFT JOIN FUS_KPI_CONTENTIEUX EK ON (EK.CONT_CHRONO_DOSSIER = CTX.CONT_CHRONO_DOSSIER)      	LEFT JOIN ( 				SELECT MAX(EPT_CHRONO) AS EPT_CHRONO,  						EPT_NATURE_PREST AS EPT_NATURE_PREST,  						CON_CHRONO AS CON_CHRONO 				FROM SIM_GCD_ENT_PREST 				GROUP BY EPT_NATURE_PREST, CON_CHRONO 				) EP ON (EP.CON_CHRONO=C.CON_CHRONO AND EP.EPT_NATURE_PREST = 'GAZ NATUREL') 	 	LEFT JOIN ( 				SELECT MAX(AVE_CHRONO) AS AVE_CHRONO,  					EPT_CHRONO AS EPT_CHRONO 				FROM SIM_GCD_DET_PREST 				GROUP BY EPT_CHRONO 				) AV ON (AV.EPT_CHRONO=EP.EPT_CHRONO)       	LEFT JOIN SIM_GCD_DET_PREST DP ON (DP.EPT_CHRONO=AV.EPT_CHRONO AND DP.AVE_CHRONO=AV.AVE_CHRONO AND (CONT_DATE_CREATION BETWEEN DP.DPT_DATE_DEBUT AND NVL(DP.DPT_DATE_FIN,to_date('31122045','ddMMyyyy'))))     LEFT JOIN SIM_GCD_TARIF TA ON (TA.TAR_CHRONO= DP.TAR_CHRONO)     LEFT JOIN SIM_GCD_DPREST_PTS DPS ON (DPS.DPT_CHRONO=DP.DPT_CHRONO)     LEFT JOIN SIM_GCD_PTS PT ON (PT.PTS_CHRONO=DPS.PTS_CHRONO)     LEFT JOIN SIM_GCT_ADR_EN_CLAIR THT ON (THT.ADRC_CHRONO=PT.PTS_ADR_CHRONO and THT.ADRC_TYP=PT.PTS_ADR_TYPE)  	 	LEFT JOIN ( 		Select SOUS_ID, 				SOUS_CLI_CHRONO_CREE, 				SOUS_CON_CHRONO_CREE, 				RANK () OVER(PARTITION BY SOUS_CLI_CHRONO_CREE, SOUS_CON_CHRONO_CREE 				ORDER BY SOUS_DATE_CREATION DESC) as RANK_SOUS 			FROM FUS_SOUSCRIPTION 			WHERE SOUS_CLI_CHRONO_CREE is not null and SOUS_CON_CHRONO_CREE is not null) SOUS ON (SOUS.SOUS_CLI_CHRONO_CREE=ACT.CLI_CHRONO and SOUS_CON_CHRONO_CREE=CTX.CONT_CHRONO_CONTRAT And RANK_SOUS=1) 			    	LEFT JOIN (SELECT  			CL.CLI_CHRONO, 			C.CON_CHRONO, 			SUM(CSO.CONSO_CONSO_ANN) AS CONSO_CONSO_ANN 		FROM FUS_PRESTATION 			INNER JOIN SIM_GCD_CONTRAT C ON (PRESTA_CON_CHRONO=C.CON_CHRONO) 			INNER JOIN SIM_GCC_CLIENTS CL ON (CL.CLI_CHRONO=C.CLI_CHRONO) 			INNER JOIN SIM_GCD_PTS PTS ON (FUS_PRESTATION.PRESTA_PTS_CHRONO=PTS.PTS_CHRONO) 			LEFT JOIN SIM_GCD_CONSOMME CSO ON (CSO.CLI_CHRONO = CL.CLI_CHRONO AND CSO.PTS_CHRONO = PTS.PTS_CHRONO) 		WHERE (CL.CLI_FAMILLE='DOM' OR CL.CLI_CATEGORIE='NDOM') and  			PRESTA_TAR_CHRONO is not null and 			sysdate between DPT_DATE_DEBUT and NVL(DPT_DATE_FIN,TO_DATE('29990101','yyyyMMdd')) 		GROUP BY CL.CLI_CHRONO,C.CON_CHRONO) CSO ON (CSO.CLI_CHRONO=ACT.CLI_CHRONO and CSO.CON_CHRONO=CTX.CONT_CHRONO_CONTRAT) 		      LEFT JOIN FUS_EXT_DOM_GCC_CLT_EMAIL EDCL ON (EDCL.CLI_CHRONO=ACT.CLI_CHRONO)   LEFT JOIN FUS_EXT_DOM_COMPTE_EMAIL  EDC ON (EDCL.CPT_EMAIL=EDC.CPT_EMAIL)  WHERE (CL.CLI_FAMILLE='DOM' OR CL.CLI_CATEGORIE='NDOM')  GROUP BY       NVL(TO_CHAR(ACT.CLI_CHRONO),'NRE'),      to_char(CTX.CONT_DATE_CREATION,'yyyyMMdd'),      'C-'||TO_CHAR(CONT_CHRONO_CONTRAT),      CTX.CONT_CHRONO_DOSSIER,      CASE WHEN CTX.CONT_ETAT_DOSSIER  =  'ENCRS' THEN 1 ELSE 0 END, 	 NVL(UPPER(ACT.CLI_E_MAIL),'NRE'), 	 NVL(TO_CHAR(CSO.CONSO_CONSO_ANN),'NRE'), 	 NVL(TO_CHAR(SOUS.SOUS_ID), 'NRE'), 	 NVL(TO_CHAR(EDC.CPT_ID), 'NRE'), 	 CASE WHEN ACT.CLI_CHRONO is not null  THEN 1 ELSE 0 END ) EKI_MKT_FACT_CONTENTIEUX"
tables = extract_tables_from_query(query)
tables2 = extract_tables_from_query(query2)
tables3 = extract_tables_from_query(query3)
print("Tables extraites (query) :", tables)
print("Tables extraites (query2) :", tables2)
print("Tables extraites (query3) :", tables3)

# Cette fonction ne donne pas les bons resultats pour la query3

Tables extraites (query) : ['T1', 'T2']
Tables extraites (query2) : ['Koudalou']
Tables extraites (query3) : ['TDI_D01', 'TDI_D02', 'TDI_D03', 'TDI_D04', 'TDI_D05', 'TDI_D06', 'TDI_D07', 'TDI_D08', 'TDI_D09', 'TDI_D10', 'TDI_D11', 'TDI_D12', 'TDI_D13', 'TDI_D14', 'TDI_D15', 'TDI_D16', 'TDI_D17', 'TDI_D18', 'TDI_D19', 'TDI_D20', 'TDI_D21', 'TDI_D22', 'TDI_D23', 'TDI_D24', 'TDI_D25', 'TDI_D26', 'TDI_D27', 'TDI_D28', 'TDI_D29', 'TDI_D30', 'TDI_I01', 'TDI_I02', 'TDI_I03', 'TDI_I04', 'TDI_I05', 'TDI_I06', 'TDI_I07', 'TDI_I08', 'TDI_I09', 'TDI_I10', 'TDI_I11', 'TDI_I12', 'TDI_I13', 'TDI_I14', 'TDI_I15', 'TDI_I16', 'TDI_I17', 'TDI_I18', 'TDI_I19', 'TDI_I20', 'EKI_MKT_FACT_CONTENTIEUX']


## Test du NLTK - echec

In [71]:
import nltk

In [66]:
from nltk import RegexpParser

In [68]:
from nltk.tokenize import word_tokenize

In [75]:
# Télécharger les données nécessaires
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\DéboraMandon\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\DéboraMandon\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping taggers\averaged_perceptron_tagger.zip.


True

In [76]:
# Fonction pour extraire les noms de tables
def extract_table_names(query):
    words = word_tokenize(query)
    tagged = nltk.pos_tag(words)

    # Définir la structure grammaticale pour les noms de tables
    grammar = r"""
        TABLE: {<DT|IN|FROM|JOIN|,>*<NN.*>+}
    """

    cp = RegexpParser(grammar)
    tree = cp.parse(tagged)

    table_names = []
    for subtree in tree.subtrees(filter=lambda t: t.label() == 'TABLE'):
        table_name = ' '.join(word for word, pos in subtree.leaves())
        table_names.append(table_name)

    return table_names

# Exemple d'utilisation
query = "SELECT * FROM employees JOIN departments ON employees.department_id = departments.department_id"
table_names = extract_table_names(query)
print("Noms de tables:", table_names)

Noms de tables: ['SELECT * FROM employees JOIN departments ON employees.department_id = departments.department_id']


In [31]:
df=pd.read_csv("sql_requetes.csv", sep=";")
df

,projet,requete_sql
0,ATP,"select * from (select nom, adresse from Immeub..."
1,ATP,SELECT * FROM Table
2,ATP,SELECT * FROM question WHERE q_id IN (SELECT r...
3,ATP,SELECT sum(a.Population)FROM travail.communes6...
4,ATP,"SELECT timestamp, log_id, severity, json_paylo..."
5,ATP,SELECT * FROM Orders WHERE CustomerID IN (SEL...
6,ATP,SELECT COUNT(count_table.mois) FROM(SELECT st....
7,PLE,"SELECT \tTDI_D01 as TDI_D01, \tTDI_D02 as TDI..."
8,PLE,"SELECT \tTDI_D01 as TDI_D01, \tTDI_D02 as TDI..."
9,PLE,"Select * FROM T1,T2 Where T1.ID=T2.FK_ID"


In [34]:
# Cette version modifiée utilise l'option '(?i)' pour rendre la recherche insensible à la casse, et permet également de gérer les virgules entre les noms de table.
df['tables'] = df['requete_sql'].apply(lambda x: set(re.findall(r'(?i)\bFROM\s+([^\s,;]+)|\bJOIN\s+([^\s,;]+)', x)[0]))

# Display the DataFrame
print(df[['projet', 'requete_sql', 'tables']])

  projet                                        requete_sql  \
0    ATP  select * from (select nom, adresse from Immeub...   
1    ATP                                SELECT * FROM Table   
2    ATP  SELECT * FROM question WHERE q_id IN (SELECT r...   
3    ATP  SELECT sum(a.Population)FROM travail.communes6...   
4    ATP  SELECT timestamp, log_id, severity, json_paylo...   
5    ATP   SELECT * FROM Orders WHERE CustomerID IN (SEL...   
6    ATP  SELECT COUNT(count_table.mois) FROM(SELECT st....   
7    PLE   SELECT \tTDI_D01 as TDI_D01, \tTDI_D02 as TDI...   
8    PLE   SELECT \tTDI_D01 as TDI_D01, \tTDI_D02 as TDI...   
9    PLE           Select * FROM T1,T2 Where T1.ID=T2.FK_ID   

                   tables  
0             {, (select}  
1               {, Table}  
2            {, question}  
3  {, travail.communes64}  
4               {, TABLE}  
5              {Orders, }  
6                   {, (}  
7                   {, (}  
8                   {, (}  
9                  {, T1} 